In [112]:
import dash
from dash import html, dcc
import plotly.express as px
from dash.dependencies import Input, Output

In [113]:
df_tips = px.data.tips()

In [114]:
df_tips.shape

(244, 7)

In [115]:
df_tips.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [116]:
df_tips.dtypes

total_bill    float64
tip           float64
sex            object
smoker         object
day            object
time           object
size            int64
dtype: object

In [117]:
df_tips.notnull().values.any()

True

In [118]:
df_tips.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [119]:
categorical_columns = ['sex', 'smoker', 'day', 'time']
numerical_columns = ['total_bill', 'tip', 'size']

In [120]:
for col in categorical_columns:
    print(f"Unique values in {col}: {df_tips[col].unique()}")

Unique values in sex: ['Female' 'Male']
Unique values in smoker: ['No' 'Yes']
Unique values in day: ['Sun' 'Sat' 'Thur' 'Fri']
Unique values in time: ['Dinner' 'Lunch']


In [121]:
# Spend by group size
group_size = df_tips.groupby("size")["total_bill"].sum().reset_index()

# Percentage tip from spending
df_tips["tip_percentage"] = (df_tips["tip"] / df_tips["total_bill"]) * 100

# Data per person
df_tips["bill_per_person"] = df_tips["total_bill"] / df_tips["size"]
df_tips["tip_per_person"] = df_tips["tip"] / df_tips["size"]

In [122]:
df_tips.head(5)

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,bill_per_person,tip_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673,8.495000,0.505000
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159,3.446667,0.553333
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734,7.003333,1.166667
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041,11.840000,1.655000
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765,6.147500,0.902500


In [123]:
df_tips.describe()

,total_bill,tip,size,tip_percentage,bill_per_person,tip_per_person
count,244.000000,244.000000,244.000000,244.000000,244.00000,244.000000
mean,19.785943,2.998279,2.569672,16.080258,7.88823,1.212762
std,8.902412,1.383638,0.951100,6.107220,2.91435,0.491705
min,3.070000,1.000000,1.000000,3.563814,2.87500,0.400000
25%,13.347500,2.000000,2.000000,12.912736,5.80250,0.862500
50%,17.795000,2.900000,2.000000,15.476977,7.25500,1.107500
75%,24.127500,3.562500,3.000000,19.147549,9.39000,1.500000
max,50.810000,10.000000,6.000000,71.034483,20.27500,3.333333


In [124]:
fig_bar = px.bar(
    group_size,
    x="size",
    y="total_bill",
    title="Total Bill by Group Size",
    labels={"size": "Group Size", "total_bill": "Total Bill (USD)"},
    template="plotly_white"
)
fig_bar.show()

In [125]:
fig_scatter = px.scatter(
    df_tips,
    x="tip_per_person",
    y="bill_per_person",
    title="Tip Per Person vs. Bill Per Person",
    labels={"tip_per_person": "Tip per Person (USD)", "bill_per_person": "Bill per Person (USD)"},
    color="sex",
    template="plotly_white"
)
fig_scatter.show()

In [126]:
# Dash app
app = dash.Dash(__name__)

# App layout 
app.layout = html.Div(children=[
    # Title
    html.H1(
        "Restaurant Dashboard",
        style={'textAlign': 'center', 'fontSize': 36, 'marginBottom': '30px', 'color': '#4a4a4a'}
    ),

    # Dropdown and slider
    html.Div(children=[
        html.Div(children=[
            html.Label("Select Gender:", style={'fontSize': 18, 'marginBottom': '10px', 'color': '#333'}),
            dcc.Dropdown(
                id="gender-dropdown",
                options=[
                    {"label": "Both", "value": "Both"},
                    {"label": "Male", "value": "Male"},
                    {"label": "Female", "value": "Female"}
                ],
                value="Both",
                clearable=False,
                style={'marginBottom': '20px'}
            ),
            html.Label("Select Day:", style={'fontSize': 18, 'marginBottom': '10px', 'color': '#333'}),
            dcc.Slider(
                id="day-slider",
                min=0,
                max=3,
                marks={0: 'Thur', 1: 'Fri', 2: 'Sat', 3: 'Sun'},
                step=None,
                value=0
            ),
        ], style={
            'padding': '20px',
            'backgroundColor': '#f1f1f1',
            'borderRadius': '8px',
            'boxShadow': '0px 2px 5px rgba(0,0,0,0.1)'
        })
    ], style={'width': '30%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '20px'}),

    # Graphs
    html.Div(children=[
        html.Div(children=[
            dcc.Graph(id="bar-chart")
        ], style={'width': '45%', 'height': '500px', 'display': 'inline-block', 'padding': '20px'}),

        html.Div(children=[
            dcc.Graph(id="scatter-chart")
        ], style={'width': '45%', 'height': '500px', 'display': 'inline-block', 'padding': '20px'}),
    ], style={'display': 'flex', 'justifyContent': 'space-around', 'paddingTop': '20px'})
], style={'padding': '20px', 'backgroundColor': '#f9f9f9'})


# Callback for chart update
@app.callback(
    [Output("bar-chart", "figure"),
     Output("scatter-chart", "figure")],
    [Input("gender-dropdown", "value"),
     Input("day-slider", "value")]
)
def update_graphs(selected_gender, selected_day):
    # Slider value per days
    day_select = {0: "Thur", 1: "Fri", 2: "Sat", 3: "Sun"}
    selected_day_name = day_select[selected_day]

    # Data filtering by gender and day
    filtered_df = df_tips[df_tips["day"] == selected_day_name]
    if selected_gender != "Both":
        filtered_df = filtered_df[filtered_df["sex"] == selected_gender]

    # Bar chart: Total spending by group size
    fig_bar = px.bar(
        filtered_df,
        x="size",
        y="total_bill",
        title=f"Total Bill by Group Size ({selected_day_name}, {selected_gender})",
        labels={"size": "Group Size", "total_bill": "Total Bill (USD)"},
        template="plotly_white"
    )

    # Scatter plot: Tips and spending per person
    fig_scatter = px.scatter(
        filtered_df,
        x="tip_per_person",
        y="bill_per_person",
        title=f"Tip Per Person vs. Bill Per Person ({selected_day_name}, {selected_gender})",
        labels={"tip_per_person": "Tip per Person (USD)", "bill_per_person": "Bill per Person (USD)"},
        color="sex",
        template="plotly_white"
    )

    return fig_bar, fig_scatter

if __name__ == '__main__':
    app.run_server(debug=True)